# Метод, основанный на биспектральном разложении сигнала

In [99]:
import os
import sys
import glob
from itertools import izip
import math

from sklearn.cross_validation import train_test_split
import cv2
from tqdm import tqdm
import pickle
import selectivesearch
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
import time
import statsmodels.stats.api as sms
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score

In [2]:
from sklearn.preprocessing import normalize
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler

In [3]:
import lasagne
import theano.tensor as T
import theano
import train_support
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

In [4]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [5]:
from sklearn.utils import shuffle

In [6]:
DEFAULT_SEED = 803

In [7]:
%matplotlib inline

In [8]:
%load_ext autoreload
%autoreload 2

## создание датафрейма из картинок

In [9]:
def create_dataframe(dir_name):
    filenames = shuffle(glob.glob(dir_name), random_state = DEFAULT_SEED)

    classes = [os.path.basename(os.path.dirname(filename)) for filename in filenames]
    patients = [filename.split("/")[-1].split("_")[0] for filename in filenames]
    indexes = [filename.split("/")[-1][:-8] for filename in filenames]
    
    class_translation = {"true":1, "false":0}
    classes = map(lambda x : class_translation[x], classes)
    
    images = np.array([cv2.imread(filename) for filename in filenames])
    
    pivot_point = (30, 210)
    image_shape = (3, 160, 160)
    
    cutimages = np.array([image[pivot_point[0]:pivot_point[0] + image_shape[1], 
                          pivot_point[1]:pivot_point[1] + image_shape[2]] for image in images])
    
    cutimages = np.array([cv2.resize(image, (80, 80)).reshape((3, 80, 80)) for image in cutimages])
    plt.imshow(cutimages[90].reshape(80, 80,3))
    images1d = cutimages.reshape(cutimages.shape[0], -1)
    images_col_names = ["".join(["imagepixel", str(i)]) for i in range(images1d.shape[1])]
    
    print len(images_col_names)
    df = pd.DataFrame(columns = images_col_names, data = images1d)
    df["ibs"] = classes
    df["Patient"] = patients
    df["ECG_ID"] = indexes
    df = shuffle(df, random_state=27)
    df = df.reset_index(drop = True)
    return df

## Функции для кросс-валидации и обучения

In [14]:
def combine_results(clf_res, cv, length):
    a = np.zeros(length)
    pr = np.zeros(length)
    
    for res, i in zip(clf_res, cv):
        itrain, itest = i
        
        a[itest] = res["p_test"]
        #pr[itest] = np.clip(res["pr_test"] - res["pr_test"].mean() + 0.5, 0.0, 1.0)
        pr[itest] = res["pr_test"]
    return a, pr

In [15]:
def cv_tt(df, target, cv):
    tt = []
    data = df.drop(target, axis = 1)
    y = df[target]
    for itrain, itest in cv:
        train_data = data.loc[itrain,:].copy()
        test_data = data.loc[itest,:].copy()
#        train_data.drop("level_0", axis=1, inplace=True)
#        test_data.drop("level_0", axis=1, inplace=True)
        train_data.reset_index(inplace=True)
        test_data.reset_index(inplace=True)
        fold = {"xtrain" : train_data, "ytrain" : y[itrain], 
                "xtest" : test_data, "ytest" : y[itest]}
        tt.append(fold)
    return tt

In [16]:
def review(df, p=None, pr=None, target="ibs", plot = True, text = True):
    lsp = np.linspace(0, 1, 201)
    
    if pr is not None:
        s = [pd.DataFrame({'label':df["Patient"], 'q':(df[target] == (pr >= l).astype(float))}).q.mean() for l in lsp]
        patients = [pd.DataFrame({'label':df["Patient"], 
                       'q':(df[target] == (pr >= l).astype(float))}).groupby('label')['q'].mean().mean() for l in lsp]
    
    if plot: 
        plt.figure(figsize = (8, 4))    
        plt.plot(lsp, patients, lw=2)
        plt.plot(lsp, s, lw=2)
        plt.xlabel('threshold')
        plt.ylabel('quality')
    if p is not None: 
        tmp = pd.DataFrame({'label':df["Patient"], 'q':df[target] == p})
    tmax = np.argmax(patients)
    
    if text:
        print ('roc-auc = ' + str(roc_auc_score(df[target], pr)))
        if p is not None:
            print ("Оценка границы разбиения по умолчанию:")      
            print ('Качество по сигналам = ' + str(tmp.q.mean()))
            print ('Качество по людям = ' + str(tmp.groupby('label')['q'].mean().mean()))
        print ("Наилучшее разбиение происходит при границе " + str(lsp[tmax]))
        print ('Качество по сигналам = ' + str(s[tmax]))
        print ('Качество по людям = ' + str(patients[tmax]))
    
    return patients[tmax]

In [17]:
def fit_predict(clf, xtrain, ytrain, xtest, ytest, clf_params, 
               fit_params, do_scaling, xgbfit, nnfit):
    if do_scaling:
        scaler = StandardScaler()
        xtrain = scaler.fit_transform(xtrain)
        xtest = scaler.transform(xtest)    
        
    if xgbfit:
        fit_params["eval_set"] = \
            [(xtrain, ytrain), (xtest, ytest)]
        fit_params["sample_weight"] = \
            np.array(get_weight(ytrain))
    if nnfit:
        fit_params["w"] = \
            np.array(get_weight(ytrain))
            
    c = clf(**clf_params)
#    c.fit(xtrain, ytrain, **fit_params)
    answer = c.fit_predict(xtrain, ytrain, xtest, ytest, **fit_params)
    
    p_tst = answer[0]
    pr_tst = answer[1]
    
#    p_tst = c.predict(xtest)
#    pr_tst = c.predict_proba(xtest)[:,1]
    
    return p_tst, pr_tst

In [18]:
def cv_fit_predict(clf, cv_tt, f_l, clf_params = {}, fit_params = {},
                   do_scaling = False, xgbfit = False, nnfit = False):
    cv_answer = []
    
    for fold in tqdm(cv_tt):
        p_tst, pr_tst = fit_predict(clf, fold["xtrain"][f_l], fold["ytrain"], 
                                      fold["xtest"][f_l], fold["ytest"], 
                                      clf_params, fit_params, do_scaling, xgbfit, nnfit)
        
        tt_answer = {"p_test" : p_tst, "pr_test" : pr_tst}
        cv_answer.append(tt_answer)
    
    return cv_answer

## Классификатор на основе нейронной сети

In [31]:
class convolutional_network_for_classifier():
    def __init__(self, image_shape, channels_number):
        self.input_var = T.tensor4('input')
        self.build_network(image_shape, channels_number)
        print "net1"
        
    def build_network(self, image_size = (180, 180), channels_number = 1):
        self.l_in = lasagne.layers.InputLayer(shape=(None, channels_number, image_size[0], image_size[1]), input_var=self.input_var)
        #self.l1_cnn = lasagne.layers.Conv2DLayer(self.l_in, num_filters=32,
        #                                filter_size=(5, 5), stride=(2, 2))
        self.l2_cnn = lasagne.layers.Conv2DLayer(self.l_in, num_filters=32,
                                        filter_size=(5, 5), stride=(2, 2))
        #self.l3_cnn = lasagne.layers.Conv2DLayer(self.l2_cnn, num_filters=32,
        #                                filter_size=(6, 6), stride=(2, 2))
        new_layer = lasagne.layers.DenseLayer(self.l2_cnn, 30, nonlinearity=lasagne.nonlinearities.leaky_rectify)
        self.l_output = lasagne.layers.DenseLayer(new_layer, 1, nonlinearity=lasagne.nonlinearities.sigmoid)

    def get_output(self):
        return self.l_output
    
    def get_input(self):
        return self.input_var

In [124]:
class CNNC():
    def __init__(self, image_shape, channels_number, lr = 0.00005, num_epochs = 50, batch_size=5):
        self.nn = convolutional_network_for_classifier(image_shape, channels_number)
        
        self.image_shape = image_shape
        self.channels_number = channels_number
        
        self.lr = lr
        self.batch_size=batch_size
        self.num_epochs = num_epochs
 
    def fit_predict(self, x_train, y_train, x_test, y_test):
        train_shape = x_train.shape
        test_shape = x_test.shape
        
        x_train = x_train.reshape(train_shape[0], self.channels_number, self.image_shape[0], self.image_shape[1])
        x_test = x_test.reshape(test_shape[0], self.channels_number, self.image_shape[0], self.image_shape[1])
        
        y_train = y_train.values
        y_test = y_test.values
        
        ret = train_support.train(self.nn.get_output(), self.nn.get_input(), 
                                  x_train, y_train, x_test, y_test,
                                  learning_rate=self.lr, num_epochs=self.num_epochs,
                                  grad_clipping=200, batch_size=self.batch_size)
        
        lasagne.layers.set_all_param_values(self.nn.get_output(), ret[0])
        
        pr = ret[4]
        
        p = np.zeros(pr.shape)
        p[pr>0.5] = 1
        
        return p, pr 
    
    def predict(self, x):
        x_shape = x.shape
        x = x.reshape(x_shape[0], self.channels_number, self.image_shape[0], self.image_shape[1])
        return train_support.predict(self.nn.get_output(), self.nn.get_input(), x)

In [125]:
def get_prediction(clf, x_train, y_train, x_test, y_test, x_val, f_l, fit_params, clf_params, do_scaling=True):
    if do_scaling:
        scaler = StandardScaler()
        x_train = scaler.fit_transform(x_train[f_l])
        x_test = scaler.transform(x_test[f_l])    
        x_val = scaler.transform(x_val[f_l])
            
    c = clf(**clf_params)

    p_tst, pr_tst = c.fit_predict(x_train, y_train, x_test, y_test, **fit_params)
    pr = c.predict(x_val)
    
    p = np.zeros(pr.shape)
    p[pr>0.5] = 1
        
    return p, pr, p_tst, pr_tst 

## Результаты на отложенной тестовой выборке (Выделение теста для остановки нейросети)

In [126]:
train_df_for_test = shuffle(train_df, random_state=10)

In [127]:
gkf = GroupKFold(n_splits=12)
cv = list(gkf.split(train_df_for_test, train_df_for_test["ibs"], groups=train_df_for_test["Patient"]))   

In [128]:
x_train = train_df_for_test.loc[cv[0][0],:].copy()
x_test = train_df_for_test.loc[cv[0][1],:].copy()


x_train.reset_index(inplace=True)
y_train = x_train["ibs"]
x_train.drop("ibs", axis=1, inplace=True)

x_test.reset_index(inplace=True)
y_test = x_test["ibs"]
x_test.drop("ibs", axis=1, inplace=True)

In [129]:
x_val = test_df.drop("ibs", axis=1)

In [137]:
cnnclf_params = {"image_shape":(80,80), "channels_number":3, "lr":0.00005, "num_epochs":100,
                 "batch_size": 6}
cnnfit_params = {}

cnn_f = train_df.columns.difference([u'ibs', u'Patient', u'ECG_ID'])

p_val, pr_val, p_test, pr_test = get_prediction(CNNC, x_train, y_train, x_test, y_test, x_val, cnn_f, cnnfit_params, cnnclf_params, do_scaling=True)

net1


  1%|          | 1/100 [00:09<14:59,  9.09s/it]

epoch 1 / 100 in 9.1s: loss 0.694296, trval_rocauc 0.757698, val_rocauc 0.675059, lr 0.000050


  2%|▏         | 2/100 [00:17<14:33,  8.91s/it]

epoch 2 / 100 in 8.5s: loss 0.696525, trval_rocauc 0.800403, val_rocauc 0.665572, lr 0.000050


...

epoch 98 / 100 in 8.5s: loss 0.682148, trval_rocauc 0.998717, val_rocauc 0.789090, lr 0.000050


 99%|█████████▉| 99/100 [14:06<00:08,  8.68s/it]

epoch 99 / 100 in 9.1s: loss 0.681437, trval_rocauc 0.996334, val_rocauc 0.784893, lr 0.000050


100%|██████████| 100/100 [14:15<00:00,  8.64s/it]

epoch 100 / 100 in 8.5s: loss 0.682525, trval_rocauc 0.997617, val_rocauc 0.782704, lr 0.000050


In [155]:
def patient_score(label, target_val, p=None):
    tmp = pd.DataFrame({'label':label, 'q':target_val == p})
    return tmp.groupby('label')['q'].mean().mean()

In [143]:
roc_auc_score(test_df["ibs"], pr_val)

0.91082631730113728

In [160]:
l = np.arange(0.1, 0.9, 0.05)
for i in l:
    tmp_p = np.zeros(pr_test.shape)
    tmp_p[pr_test>i] = 1
    print "i=", i, " score =" , patient_score(x_test["Patient"], y_test, tmp_p.reshape(-1))

i= 0.1  score = 0.536585365854
i= 0.15  score = 0.536585365854
i= 0.2  score = 0.536585365854
i= 0.25  score = 0.536585365854
i= 0.3  score = 0.536585365854
i= 0.35  score = 0.538617886179
i= 0.4  score = 0.624153116531
i= 0.45  score = 0.733883156139
i= 0.5  score = 0.566056910569
i= 0.55  score = 0.481199186992
i= 0.6  score = 0.463414634146
i= 0.65  score = 0.463414634146
i= 0.7  score = 0.463414634146
i= 0.75  score = 0.463414634146
i= 0.8  score = 0.463414634146
i= 0.85  score = 0.463414634146


In [166]:
p_t = np.zeros(pr_val.shape)
p_t[pr_val>0.45] = 1

In [167]:
accuracy_score(test_df["ibs"], p_t)

0.84875558391831529

In [168]:
f1_score(test_df["ibs"], p_t)

0.79656652360515012

In [169]:
patient_score(test_df["Patient"], test_df["ibs"], p_t.reshape(-1))

0.7432526815623264

In [172]:
r_df = pd.DataFrame(data={"ibs":test_df["ibs"], "ECG_ID":test_df["ECG_ID"], "CNN_PR":pr_val.reshape(-1), 
                          "CNN_P":p_t.reshape(-1)})

In [174]:
r_df.to_csv("cnn_test_results.csv")

## Результаты на кросс-валидации

In [33]:
gkf = GroupKFold(n_splits=12)
cv = list(gkf.split(train_df, train_df["ibs"], groups=train_df["Patient"]))    

In [34]:
tt = cv_tt(train_df, "ibs", cv)

In [35]:
cnn_f = train_df.columns.difference([u'ibs', u'Patient', u'ECG_ID'])

In [306]:
cnnclf_params = {"image_shape":(80,80), "channels_number":3, "lr":0.00005, "num_epochs":100,
                 "batch_size": 6}
cnnfit_params = {}
cnn_res = cv_fit_predict(CNNC, tt, cnn_f, cnnclf_params, cnnfit_params, do_scaling = True)

  1%|          | 1/100 [00:03<05:54,  3.58s/it]

epoch 1 / 100 in 3.6s: loss 0.696675, trval_rocauc 0.632081, val_rocauc 0.722041, lr 0.000050



  2%|▏         | 2/100 [00:07<05:50,  3.57s/it]

epoch 2 / 100 in 3.6s: loss 0.694834, trval_rocauc 0.701313, val_rocauc 0.717462, lr 0.000050



  3%|▎         | 3/100 [00:10<05:45,  3.56s/it]

...

epoch 98 / 100 in 3.3s: loss 0.678535, trval_rocauc 1.000000, val_rocauc 0.643297, lr 0.000050



 99%|█████████▉| 99/100 [05:21<00:03,  3.24s/it]

epoch 99 / 100 in 3.2s: loss 0.680797, trval_rocauc 1.000000, val_rocauc 0.635326, lr 0.000050



100%|██████████| 12/12 [1:08:08<00:00, 329.96s/it]

epoch 100 / 100 in 3.2s: loss 0.678899, trval_rocauc 1.000000, val_rocauc 0.626993, lr 0.000050


In [322]:
cnn_p, cnn_pr = combine_results(cnn_res, cv, df.shape[0])

/Users/GoormanMcBook/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.
/Users/GoormanMcBook/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.


In [411]:
def review(df, p=None, pr=None, target="ibs", plot = True, text = True):
    if p is not None: 
        tmp = pd.DataFrame({'label':df["Patient"], 'q':df[target] == p})
    
    if text:
        print ('roc-auc = ' + str(roc_auc_score(df[target], pr)))
        if p is not None:     
            print ('Качество по сигналам = ' + str(tmp.q.mean()))
            print ('Качество по людям = ' + str(tmp.groupby('label')['q'].mean().mean()))

In [417]:
def combine_results_patient(df, clf_res, cv, length):
    a = np.zeros(length)
    pr = np.zeros(length)
    scores = []
    for res, i in zip(clf_res, cv):
        itrain, itest = i
        pr_s = np.arange(0.4, 0.7, 0.01)
        best_score = 0
        best_q = -5
        best_p = None
        res_pred = res["pr_test"].reshape(-1)#(res["pr_test"] - res["pr_test"].mean() + 0.5).reshape(-1)
        for q in pr_s:
            test_df = df.loc[itest, :]
            temp_res = np.zeros(len(test_df["ibs"]))
            temp_res[res["pr_test"].reshape(-1) >= q] = 1.0
            score = pd.DataFrame({'label':test_df["Patient"], 
                       'q':(test_df["ibs"] == (res_pred >= q).astype(float))}).groupby('label')['q'].mean().mean()
            if score > best_score:
                best_score = score
                best_q = q
                best_p = temp_res.copy()
        print best_score, " ", best_q
        scores.append(best_score)
        
        a[itest] = best_p
        pr[itest] = res["pr_test"]
    print np.mean(scores)
    print scores
    return a, pr  

In [419]:
def combine_results(df, clf_res, cv, length):
    a = np.zeros(length)
    pr = np.zeros(length)
    scores = []
    for res, i in zip(clf_res, cv):
        itrain, itest = i
        pr_s = np.arange(0.4, 0.7, 0.01)
        best_score = 0
        best_q = -5
        best_p = None
        res_pred = res["pr_test"].reshape(-1)#(res["pr_test"] - res["pr_test"].mean() + 0.5).reshape(-1)
        for q in pr_s:
            test_df = df.loc[itest, :]
            temp_res = np.zeros(len(test_df["ibs"]))
            temp_res[res["pr_test"].reshape(-1) >= q] = 1.0
            score = accuracy_score(test_df["ibs"],  temp_res)
            if score > best_score:
                best_score = score
                best_q = q
                best_p = temp_res.copy()
        print best_score, " ", best_q
        scores.append(best_score)
        
        a[itest] = best_p
        pr[itest] = res["pr_test"]
    print np.mean(scores)
    print scores
    return a, pr  

In [416]:
cnn_p_p, cnn_pr_p = combine_results_patient(df, cnn_res, cv, df.shape[0])

0.926666666667   0.59
0.86   0.54


/Users/GoormanMcBook/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:25: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.


0.846666666667   0.52
0.86   0.4
0.866666666667   0.49
0.766666666667   0.46
0.753333333333   0.51
0.673333333333   0.48
0.626666666667   0.5
0.666666666667   0.43
0.637583892617   0.47
0.657718120805   0.44
0.761830723341
[0.92666666666666664, 0.85999999999999999, 0.84666666666666668, 0.85999999999999999, 0.8666666666666667, 0.76666666666666672, 0.7533333333333333, 0.67333333333333334, 0.62666666666666671, 0.66666666666666663, 0.63758389261744963, 0.65771812080536918]


In [430]:
cnn_p, cnn_pr = combine_results(df, cnn_res, cv, df.shape[0])

0.926666666667   0.59
0.86   0.54


/Users/GoormanMcBook/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:25: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.


0.846666666667   0.52
0.86   0.4
0.866666666667   0.49
0.766666666667   0.46
0.753333333333   0.51
0.673333333333   0.48
0.626666666667   0.5
0.666666666667   0.43
0.637583892617   0.47
0.657718120805   0.44
0.761830723341
[0.92666666666666664, 0.85999999999999999, 0.84666666666666668, 0.85999999999999999, 0.8666666666666667, 0.76666666666666672, 0.7533333333333333, 0.67333333333333334, 0.62666666666666671, 0.66666666666666663, 0.63758389261744963, 0.65771812080536918]


In [463]:
review(df, cnn_p, cnn_pr, "ibs", plot=True, text=True)

roc-auc = 0.741881573996
Качество по сигналам = 0.761957730812
Качество по людям = 0.669264078247


In [464]:
review(df, cnn_p_p, cnn_pr_p, "ibs", plot=True, text=True)

roc-auc = 0.741881573996
Качество по сигналам = 0.687986651835
Качество по людям = 0.718565169076


In [467]:
new_df = pd.DataFrame(data={"ibs":df["ibs"], "ECG_ID":df["ECG_ID"], "CNN_PR":cnn_pr, "CNN_P":cnn_p, "CNN_PRP":cnn_p_p})

In [468]:
new_df.to_csv("cnn_results.csv")